In [1]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.Util.Padding import pad, unpad
import urllib3
import json
import sys
import base64
import requests


In [2]:
base_url = "http://127.0.0.1:8000"

In [3]:

response = requests.get(f"{base_url}/getKey").json()
server_public_key = response['public_key']

print(response['public_key'])

-----BEGIN PUBLIC KEY-----
MIICIjANBgkqhkiG9w0BAQEFAAOCAg8AMIICCgKCAgEAjIPz3MwO7O2NSu3n1vrS
g+F8L3hmLH+gsCM0pwvLQo8RGGw/01rtnAjrnOoFECmVHNEPJwE14286crxYySqQ
KYp7g9fCl8X8fKalSRtmwOPT7qjd/drYxNdAhvYrvzBybjXvKw7bZz8XPbqChNyO
3oPz+QZ52/BMiL4W+gtpzhSCQDfn5qVP6zrzy+yzVR38r9O25TZVmcuC5OUB9lQn
qaJF2wHs4lgZOGJ967I5a4P50JOhLdFlEfjs2sWgngWXBINubEaR1LcBoyvwOszz
j6ZWZdn/IUfCUYtbhCoTS29C5Eg6daM7+qDg7nNdHQp5TSBIiBRaUjXP3Jgbr2a0
78rXBjbJJ53MrYCzGKJreovlKIR9CA7yo8EfMTKEXSiZkPq9wUOF6HmYvqXs1oBa
qXjp33Nv6a79yvPuAEKg86Y3iyPqSCy4hA4W+nrh8g1ZJeta2JQfEE/hJUC1Xbf1
AnQjjpgQ+BrYJZYO3UPv4mhuEmvdIS50uXBmHjM23HB7WhPopVcmeKniDOg85okh
qjq5qQ4nNMnGhcq7eDdi81F8UzSPuCPSxwvL08Es0AV1U7HIGmEUyCTGP6fTyO5O
U5CDC8b7SkrstNZ5Re7mp4Rksb/cWHbhOYXBdX00PeuAZHeyIp0JozG/Rvh9SSEH
6HEYxb3eg+nkep9J0L4I8YMCAwEAAQ==
-----END PUBLIC KEY-----


In [4]:
key = RSA.generate(2048)

In [5]:
"""AES """
key = b'Sixteen byte key'

In [6]:
"""Encrypts the AES encryption key"""
rsa_cipher = PKCS1_OAEP.new(RSA.import_key(server_public_key))
encrypted_key = rsa_cipher.encrypt(key)
# print(encrypted_key)

In [7]:
message = {
    "foo": "bar",
    "hello": "hello",
    "random": "random"
}
data_str = json.dumps(message)

In [8]:
"""Encryption the message data"""
cipher = AES.new(key, AES.MODE_CBC)
ciphertext = cipher.encrypt(pad(data_str.encode(), AES.block_size))

In [9]:
request_data = {
    "message" : str(base64.b64encode(ciphertext), 'utf-8'),
    "encrypted_key" : str(base64.b64encode(encrypted_key), 'utf-8'),
    "initialization_vector": str(base64.b64encode(cipher.iv), 'utf-8'), 
}


In [10]:
response = requests.post(f"{base_url}/message", json=request_data)

In [11]:
response_dict = json.loads(response.json())


response_dict


{'foo': 'bar', 'hello': 'hello', 'random': 'random'}

In [12]:
for x in message.keys():
    print(response_dict[x])

bar
hello
random


'random'